# How to Visualize Custom TFX Artifacts With InteractiveContext

This notebook is a companion to a [blog post by Suzen Fylke](https://codesue.com/blog/how-to-visualize-custom-tfx-artifacts-with-interactivecontext/).

## Setup

Install TFX, verify the installation, then restart the notebook kernel.

In [5]:
%%capture
! pip install tfx

In [6]:
import tfx
tfx.__version__

'1.9.1'

In [7]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

## Create a custom artifact, component, and visualizer

Here we create a custom `Card` artifact, a `create_greeting_card` component that produces the artifact, and a `CardVisualization` that we can use to display the artifact with `InteractiveContext`.

In [8]:
%%writefile custom_components.py

from pathlib import Path
from textwrap import dedent

from tfx.dsl.component.experimental.decorators import component
from tfx.dsl.component.experimental.annotations import OutputArtifact, Parameter
from tfx.orchestration.experimental.interactive.visualizations import ArtifactVisualization
from tfx.types.artifact import Artifact

class Card(Artifact):
  TYPE_NAME = 'Card'

@component
def create_greeting_card(
  greeting: Parameter[str],
  card: OutputArtifact[Card],
):
  card_template = dedent(
    '''
      <!DOCTYPE html>
      <html>
        <head>
          <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Montserrat">
          <style>
            body {{
              border: 1px solid #e6e9ef;
              border-radius: 0.75rem;
            }}

            .container {{
              display: grid;
              place-items: center;  
              text-align: center;
            }}

            .greeting {{
              --color-1: #DF8453;
              --color-2: #3D8DAE;
              --color-3: #E4A9A8;
              animation: color-animation 4s linear infinite;
              font-family: "Montserrat", sans-serif;
              font-weight: 800;
              font-size: 8.5vw;
              text-transform: uppercase;
            }}

            @keyframes color-animation {{
              0%    {{color: var(--color-1)}}
              32%   {{color: var(--color-1)}}
              33%   {{color: var(--color-2)}}
              65%   {{color: var(--color-2)}}
              66%   {{color: var(--color-3)}}
              99%   {{color: var(--color-3)}}
              100%  {{color: var(--color-1)}}
            }}
          </style>
        </head>
        <body>
          <div class="container">
            <h2><span class="greeting">{greeting}</span></h2>
          </div>
        </body>
      </html>
    '''
  )
  p = Path(card.uri) / 'greeting.html'
  p.write_text(card_template.format(greeting=greeting))


class CardVisualization(ArtifactVisualization):

  ARTIFACT_TYPE = Card

  def display(self, artifact: Artifact):
    from IPython.display import display, HTML
    files = Path(artifact.uri).glob('*.html')
    card = next(files).read_text()
    display(HTML(card))

Overwriting custom_components.py


## Visualize the custom artifact with InteractiveContext

Now, let's visualize the custom artifact.

In [9]:
from custom_components import create_greeting_card, CardVisualization
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.experimental.interactive import visualizations

context = InteractiveContext()
visualizations.get_registry().register(CardVisualization)

greeting_card = create_greeting_card(greeting='Hejsan!')
context.run(greeting_card)
context.show(greeting_card.outputs['card'])